In [ ]:
import pandas as pd
from google.colab import drive

#Load 'all' CSV file
drive.mount('/content/drive', force_remount=True)
file_path = '/content/drive/MyDrive/all.csv'

data = pd.read_csv(file_path)

# data.head(), data.columns

#Name columns for better understanding
columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Index Name']
data.columns = columns

#Convert the 'Datetime' column to datetime type
data['Datetime'] = pd.to_datetime(data['Datetime'], errors='coerce')

#Strip any file extension from 'Index Name' to keep it clean
data['Index Name'] = data['Index Name'].str.replace('.csv', '', regex=False)

data.head(), data.dtypes


Mounted at /content/drive


(                   Datetime      Open      High       Low     Close  Volume  \
 0 2023-01-02 09:16:00+05:30  43135.40  43151.25  43097.20  43118.30       0   
 1 2023-01-02 09:17:00+05:30  43116.50  43120.95  43071.55  43105.70       0   
 2 2023-01-02 09:18:00+05:30  43108.20  43108.60  43061.40  43077.45       0   
 3 2023-01-02 09:19:00+05:30  43076.55  43091.65  43030.05  43030.05       0   
 4 2023-01-02 09:20:00+05:30  43044.60  43069.45  42987.45  43056.05       0   
 
     Index Name  
 0  0 BANKNIFTY  
 1  0 BANKNIFTY  
 2  0 BANKNIFTY  
 3  0 BANKNIFTY  
 4  0 BANKNIFTY  ,
 Datetime      datetime64[ns, UTC+05:30]
 Open                            float64
 High                            float64
 Low                             float64
 Close                           float64
 Volume                            int64
 Index Name                       object
 dtype: object)

In [ ]:
#Clean up the Name
data['Index Name'] = data['Index Name'].str.replace('^[0-9]+ ', '', regex=True)

# Split the data into separate DataFrames for each index
index_dataframes = {index: df.sort_values('Datetime') for index, df in data.groupby('Index Name')}

# Check a sample from one of the dataframes to verify
index_dataframes[list(index_dataframes.keys())[0]].head()


,Datetime,Open,High,Low,Close,Volume,Index Name
0,2023-01-02 09:16:00+05:30,43135.40,43151.25,43097.20,43118.30,0,BANKNIFTY
1,2023-01-02 09:17:00+05:30,43116.50,43120.95,43071.55,43105.70,0,BANKNIFTY
2,2023-01-02 09:18:00+05:30,43108.20,43108.60,43061.40,43077.45,0,BANKNIFTY
3,2023-01-02 09:19:00+05:30,43076.55,43091.65,43030.05,43030.05,0,BANKNIFTY
4,2023-01-02 09:20:00+05:30,43044.60,43069.45,42987.45,43056.05,0,BANKNIFTY


In [ ]:
#Dictionary to store each DataFrame by index name
index_df = {}

# Split the data and store each in the dictionary
for index_name, group_df in data.groupby('Index Name'):
    index_df[index_name] = group_df.sort_values('Datetime').drop('Volume', axis=1)

#Now you can access each DataFrame by its index name, e.g., index_dataframes['YourIndexName']


In [ ]:
midcap_df = index_df['MIDCPNIFTY'].copy()
banknifty_df = index_df['BANKNIFTY'].copy()
finnifty_df = index_df['FINNIFTY'].copy()
nifty_df = index_df['NIFTY'].copy()

In [ ]:
import pandas as pd

def calculate_returns(df):
    #Calculate returns for different time intervals: 1 minute, 5 minutes, and 10 minutes
    #Shift the 'Close' prices by 1, 5, and 10 rows respectively
    df['Shifted_Close_1'] = df['Close'].shift(1)
    df['Shifted_Close_5'] = df['Close'].shift(5)
    df['Shifted_Close_10'] = df['Close'].shift(10)

    #Calculate the returns: (Current Close - Shifted Close) / Shifted Close, times 100*100 for basis points
    df['1_Min_Returns'] = ((df['Close'] - df['Shifted_Close_1']) / df['Shifted_Close_1']) * 10000
    df['5_Min_Returns'] = ((df['Close'] - df['Shifted_Close_5']) / df['Shifted_Close_5']) * 10000
    df['10_Min_Returns'] = ((df['Close'] - df['Shifted_Close_10']) / df['Shifted_Close_10']) * 10000

    #Drop the temporary 'Shifted_Close' columns, no longer needed
    df.drop(columns=['Shifted_Close_1', 'Shifted_Close_5', 'Shifted_Close_10'], inplace=True)

    return df


In [ ]:
midcap_df = calculate_returns(midcap_df)
banknifty_df = calculate_returns(banknifty_df)
finnifty_df = calculate_returns(finnifty_df)
nifty_df = calculate_returns(nifty_df)

In [ ]:
len(midcap_df), len(banknifty_df), len(finnifty_df), len(nifty_df)

(145376, 145375, 145376, 145376)

In [ ]:
import pandas as pd

# Ensure Datetime is the index for each DataFrame
midcap_df.set_index('Datetime', inplace=True)
banknifty_df.set_index('Datetime', inplace=True)
finnifty_df.set_index('Datetime', inplace=True)
nifty_df.set_index('Datetime', inplace=True)

In [ ]:
#TO NOTE: Banknifty does not have a row for 9:15am on 2023-01-02
# midcap_df.iloc[0]
midcap_df = midcap_df.drop('2023-01-02 09:15:00+05:30')
finnifty_df = finnifty_df.drop('2023-01-02 09:15:00+05:30')
nifty_df = nifty_df.drop('2023-01-02 09:15:00+05:30')

In [ ]:
len(midcap_df), len(banknifty_df), len(finnifty_df), len(nifty_df)

(145375, 145375, 145375, 145375)

In [ ]:
#Merging returns from other DataFrames into banknifty_df, and so on

banknifty_df['1_Min_Returns_Midcap'] = midcap_df['1_Min_Returns']
banknifty_df['5_Min_Returns_Midcap'] = midcap_df['5_Min_Returns']
banknifty_df['10_Min_Returns_Midcap'] = midcap_df['10_Min_Returns']

banknifty_df['1_Min_Returns_Finnifty'] = finnifty_df['1_Min_Returns']
banknifty_df['5_Min_Returns_Finnifty'] = finnifty_df['5_Min_Returns']
banknifty_df['10_Min_Returns_Finnifty'] = finnifty_df['10_Min_Returns']

banknifty_df['1_Min_Returns_Nifty'] = nifty_df['1_Min_Returns']
banknifty_df['5_Min_Returns_Nifty'] = nifty_df['5_Min_Returns']
banknifty_df['10_Min_Returns_Nifty'] = nifty_df['10_Min_Returns']


# Merging returns from other dataframes into finnifty df

In [ ]:
# Merging returns from other DataFrames into finnifty_df
finnifty_df['1_Min_Returns_Midcap'] = midcap_df['1_Min_Returns']
finnifty_df['5_Min_Returns_Midcap'] = midcap_df['5_Min_Returns']
finnifty_df['10_Min_Returns_Midcap'] = midcap_df['10_Min_Returns']

finnifty_df['1_Min_Returns_Nifty'] = nifty_df['1_Min_Returns']
finnifty_df['5_Min_Returns_Nifty'] = nifty_df['5_Min_Returns']
finnifty_df['10_Min_Returns_Nifty'] = nifty_df['10_Min_Returns']

finnifty_df['1_Min_Returns_Banknifty'] = banknifty_df['1_Min_Returns']
finnifty_df['5_Min_Returns_Banknifty'] = banknifty_df['5_Min_Returns']
finnifty_df['10_Min_Returns_Banknifty'] = banknifty_df['10_Min_Returns']

In [ ]:
midcap_df['1_Min_Returns_Finnifty'] = finnifty_df['1_Min_Returns']
midcap_df['5_Min_Returns_Finnifty'] = finnifty_df['5_Min_Returns']
midcap_df['10_Min_Returns_Finnifty'] = finnifty_df['10_Min_Returns']

midcap_df['1_Min_Returns_Nifty'] = nifty_df['1_Min_Returns']
midcap_df['5_Min_Returns_Nifty'] = nifty_df['5_Min_Returns']
midcap_df['10_Min_Returns_Nifty'] = nifty_df['10_Min_Returns']

midcap_df['1_Min_Returns_Banknifty'] = banknifty_df['1_Min_Returns']
midcap_df['5_Min_Returns_Banknifty'] = banknifty_df['5_Min_Returns']
midcap_df['10_Min_Returns_Banknifty'] = banknifty_df['10_Min_Returns']


In [ ]:
save_path = '/content/drive/My Drive/wealthnet_features_returns/midcap.csv'
midcap_df.to_csv(save_path)

In [ ]:
# result_df.iloc[92000:92020]
# result_df.shape
banknifty_df.head()

,Open,High,Low,Close,Index Name,1_Min_Returns,5_Min_Returns,10_Min_Returns,1_Min_Returns_Midcap,5_Min_Returns_Midcap,10_Min_Returns_Midcap,1_Min_Returns_Finnifty,5_Min_Returns_Finnifty,10_Min_Returns_Finnifty,1_Min_Returns_Nifty,5_Min_Returns_Nifty,10_Min_Returns_Nifty
Datetime,,,,,,,,,,,,,,,,,
2023-01-02 09:16:00+05:30,43135.40,43151.25,43097.20,43118.30,BANKNIFTY,NaN,NaN,NaN,-6.862657,NaN,NaN,-3.996655,NaN,NaN,-3.141993,NaN,NaN
2023-01-02 09:17:00+05:30,43116.50,43120.95,43071.55,43105.70,BANKNIFTY,-2.922193,NaN,NaN,-6.798002,NaN,NaN,4.471731,NaN,NaN,2.701861,NaN,NaN
2023-01-02 09:18:00+05:30,43108.20,43108.60,43061.40,43077.45,BANKNIFTY,-6.553658,NaN,NaN,-0.555316,NaN,NaN,-8.071810,NaN,NaN,-5.402262,NaN,NaN
2023-01-02 09:19:00+05:30,43076.55,43091.65,43030.05,43030.05,BANKNIFTY,-11.003437,NaN,NaN,-12.009385,NaN,NaN,-8.736175,NaN,NaN,-9.734843,NaN,NaN
2023-01-02 09:20:00+05:30,43044.60,43069.45,42987.45,43056.05,BANKNIFTY,6.042289,NaN,NaN,3.961607,-22.251645,NaN,1.448523,-14.882283,NaN,2.622411,-12.953832,NaN


In [ ]:
midcap_df = midcap_df.join([
    banknifty_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_BANKNIFTY'),
    finnifty_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_FINNIFTY'),
    nifty_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_NIFTY')
], how='left')



In [ ]:
finnifty_df = finnifty_df.join([
    midcap_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_MIDCAP'),
    banknifty_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_BANKNIFTY'),
    nifty_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_NIFTY')
], how='left')



In [ ]:
nifty_df = nifty_df.join([
    midcap_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_MIDCAP'),
    banknifty_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_BANKNIFTY'),
    finnifty_df[['1_Min_Returns', '5_Min_Returns', '10_Min_Returns']].add_suffix('_FINNIFTY')
], how='left')